In [59]:
import torch
import torchaudio
import os
import numpy as np
import pandas as pd
import json

DIR = '../audioData/sythenticSoundscenes/test/'
dir_tut = '../audioData/TUTUrban2018/developmentDataset/TUT-urban-acoustic-scenes-2018-development/'

In [60]:
tut_csv = os.path.join(dir_tut,'train.csv')
tut_df = pd.read_csv(tut_csv)
tut_df

,files,labels
0,audio/airport-barcelona-0-0-a.wav,airport
1,audio/airport-barcelona-0-1-a.wav,airport
2,audio/airport-barcelona-0-2-a.wav,airport
3,audio/airport-barcelona-0-3-a.wav,airport
4,audio/airport-barcelona-0-4-a.wav,airport
...,...,...
6117,audio/tram-vienna-202-6117-a.wav,tram
6118,audio/tram-vienna-202-6118-a.wav,tram
6119,audio/tram-vienna-202-6119-a.wav,tram
6120,audio/tram-vienna-202-6120-a.wav,tram


In [ ]:
all_files = os.listdir('../audioData/sythenticSoundscenes/test/')
aud_file_list = []

for file in all_files:
    if file.split('.')[-1] == 'wav':
        aud_file_list.append(file)



In [ ]:
lab_file_list = []
for file in all_files:
    if file.split('.')[-1] == 'jams':
        lab_file_list.append(file)

In [ ]:
from torch.utils.data import Dataset

class scraperDataset(Dataset):

    def __init__(self, dataset_csv, data_dir, only_scene=False):

        self.dataset_csv = dataset_csv
        self.data_directory = data_dir
        self.only_scene = only_scene
        self.dataframe = pd.read_csv(dataset_csv)

        self.audio_files = self.dataframe['audio_fileNames']
        self.label_files = self.dataframe['label_fileNames']
        self.scene_labels = self.dataframe['acoustic_scene_label']
        self.events_label_list = self.dataframe['events_label_list']

    def __len__(self):
        return len(self.audio_files)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Load Audio file
        audio_file = os.path.join(self.data_directory, self.audio_files[idx])
        audio_data, sr = torchaudio.load(audio_file)

        if self.only_scene:
            sample = {'audio':audio_data, 'scene_label':self.scene_labels[idx]}
        else:
            sample = {'audio':audio_data, 'scene_label':self.scene_labels[idx], 'event_list':self.events_label_list[idx]}
        
        return sample


In [ ]:
file_count = pd.DataFrame()
file_count['audio_fileNames'] = aud_file_list
file_count['label_fileNames'] = lab_file_list


In [ ]:
sam_file = '../audioData/sythenticSoundscenes/test/restaurant8_downshift_0.wav'

def get_label_data_json(audio_filename):
    dir_name, fname = os.path.split(audio_filename)
    lab_fname = os.path.join(dir_name, (fname.split('.')[0] + '.jams'))
    with open(lab_fname, 'r') as f:
        label_data_json = json.load(f)

    return label_data_json

def get_justLabels(audio_filename, event_labels=True):
    scene_label = []
    event_label = []
    label_data_json = get_label_data_json(audio_filename)
    
    for events in label_data_json['annotations'][0]['data']:
        event_label.append(events['value']['label'])
    scene_label.append(event_label.pop(0))
    #print(scene_label, event_label)
    if event_labels:
        return scene_label, event_label
    else:
        return scene_label

def get_labels_with_timestamps(audio_filename):
    events_dict = {}
    label_data_json = get_label_data_json(audio_filename)

    for events in label_data_json['annotations'][0]['data']:
        start_time = events['value']['event_time']
        duration = events['value']['event_duration']
        events_dict[events['value']['label']] = [start_time, duration]

    return events_dict

get_justLabels(sam_file, event_labels=False)

In [ ]:
activity_labels_list = []
for file in aud_file_list:
    activity_labels_list.append(get_justLabels(os.path.join(DIR,file), event_labels=False)[0])

In [ ]:
import re

pattern = re.compile(r'\d+')

for i in range(0,len(activity_labels_list)):
    activity_labels_list[i] = pattern.sub('', activity_labels_list[i])

activity_labels_list[-1]

In [ ]:
file_count['acoustic_scene_label'] = activity_labels_list
file_count

In [ ]:
event_label_list = []
for file in aud_file_list:
    event_label_list.append(get_justLabels(os.path.join(DIR, file), True)[1])

len(event_label_list)

In [ ]:
file_count['events_label_list'] = event_label_list
file_count

In [ ]:
file_count.to_csv('Datasets/scrapper_dataset.csv', index=False)

In [ ]:
file_count['acoustic_scene_label'].unique()

In [ ]:
sam_file = '../audioData/sythenticSoundscenes/test/bus1_0.jams'

with open(sam_file, 'r') as f:
    data = json.load(f)


#data = json.dumps(data)
for i in data['annotations'][0]['data']:
    print(i['value']['label'])

In [ ]:
data